In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
%matplotlib inline


import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
# create element tree object 
tree = ET.parse('./export.xml') 

# for every health record, extract the attributes into a dictionary (columns). Then create a list (rows).
root = tree.getroot()
record_list = [x.attrib for x in root.iter('Record')]

# create DataFrame from a list (rows) of dictionaries (columns)
data = pd.DataFrame(record_list)

# proper type to dates
for col in ['creationDate', 'startDate', 'endDate']:
    data[col] = pd.to_datetime(data[col])

# value is numeric, NaN if fails
data['value'] = pd.to_numeric(data['value'], errors='coerce')

# some records do not measure anything, just count occurences
# filling with 1.0 (= one time) makes it easier to aggregate
data['value'] = data['value'].fillna(1.0)

# shorter observation names: use vectorized replace function
data['type'] = data['type'].str.replace('HKQuantityTypeIdentifier', '')
data['type'] = data['type'].str.replace('HKCategoryTypeIdentifier', '')

In [106]:
pivot_df = data.pivot_table(index='endDate', columns='type', values='value')
df = pivot_df.resample('1min').mean()
max(df.WalkingSpeed.dropna())

8.208

In [107]:

# fig = plt.figure(figsize=(16,12)) 
# x = np.arange(df['HeartRateVariabilitySDNN'].dropna().index.size)
# y = df['HeartRateVariabilitySDNN'].dropna().values
# sns.regplot(x=x, y=y, scatter=False)
# ax=sns.lineplot(x=x, y=y,err_style="bars", ci='sd')
# ax.set(xlabel='Observation in time', ylabel='HeartRateVariabilitySDNN')


#x = np.arange(df.columns[15:17].dropna().index.size)
x= df[df.columns[15:17]].dropna().values
y = np.where((df["BloodGlucose"].dropna().values< 3.9), 0, 1)


y

array([0, 1, 1, ..., 1, 1, 0])

In [108]:
# fig = plt.figure(figsize=(16,12)) 
# x = np.arange(df['BloodGlucose'].dropna().index.size)
# y = df['BloodGlucose'].dropna().values
# sns.regplot(x=x, y=y, scatter=False)
# ax=sns.lineplot(x=x, y=y,err_style="bars", ci='sd')
# ax.set(xlabel='Observation in time', ylabel='BloodGlucose')

In [109]:

ml_df = pd.merge_asof(
    df[df.columns[15:17]].dropna(),
    df['BloodGlucose'].dropna(),
    left_index=True,
    right_index=True,
    direction="nearest",
)



In [110]:
ml_df[ml_df["BloodGlucose"]<4.0]

HeartRate  HeartRateVariabilitySDNN  BloodGlucose
endDate                                                                     
2021-12-25 05:33:00+02:00    47.0000                   62.3037       2.16479
2021-12-27 08:48:00+02:00    58.6897                   83.1272       3.99654
2021-12-28 16:38:00+02:00    63.4322                  100.8650       2.77537
2022-02-21 21:03:00+02:00    57.0000                   36.0798       3.77451
2022-03-08 15:06:00+02:00    66.4935                   48.2973       2.70000
2022-03-11 09:47:00+02:00    83.0000                   99.0896       2.70000
2022-03-11 17:57:00+02:00    74.2029                  147.7820       2.70000
2022-03-16 16:38:00+02:00    71.0000                   60.1067       2.70000
2022-03-16 19:28:00+02:00    85.7839                  153.8330       2.70000
2022-03-16 23:07:00+02:00    80.9751                   81.1904       2.70000
2022-03-18 08:41:00+02:00    57.7444                   38.0936       2.70000
2022-03-18 13:10:00+02:00    80.0000                   99.3812       2.70000
2022-03-18 16:38:00+02:00    62.0000                   59.0387       2.70000

In [111]:
# x_2 = df[df.columns[15:17]]
# x_2.dropna()
ml_df[ml_df["BloodGlucose"]>4.0]

HeartRate  HeartRateVariabilitySDNN  BloodGlucose
endDate                                                                      
2021-12-20 14:19:00+02:00   71.972500                   80.7055        5.7000
2021-12-20 14:58:00+02:00   50.662300                   67.4926        5.7000
2021-12-20 15:48:00+02:00   61.178850                   89.7570        5.7000
2021-12-20 18:28:00+02:00  110.800375                   15.1997        5.7000
2021-12-20 19:57:00+02:00  102.496000                   24.1745        5.7000
...                               ...                       ...           ...
2022-03-05 15:25:00+02:00   64.810100                   29.2457       10.4354
2022-03-06 07:58:00+02:00   56.000000                  157.8630       10.4354
2022-03-06 09:41:00+02:00   55.000000                   92.4392       10.4354
2022-03-08 09:29:00+02:00   64.267800                   60.5749       10.4354
2022-03-08 13:05:00+02:00   70.000000                   92.4256       10.4354

[168 rows x 3 columns]

In [112]:
# from sklearn.svm import SVR
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.pipeline import make_pipeline
# from sklearn.preprocessing import StandardScaler
# import numpy as np


# regr = make_pipeline(StandardScaler(), RandomForestClassifier())

    
    
# x = ml_df["HeartRateVariabilitySDNN"].values
# x = x.reshape(len(x), 1)
# y = np.where((df["BloodGlucose"].dropna().values< 3.9), 0, 1)
# svr = regr.fit(x, y)
# svr.score(x,y)
x

array([[ 71.9725    ,  80.7055    ],
       [ 50.6623    ,  67.4926    ],
       [ 61.17885   ,  89.757     ],
       [110.800375  ,  15.1997    ],
       [102.496     ,  24.1745    ],
       [ 79.4651    ,  67.9572    ],
       [ 72.91635   ,  52.8636    ],
       [ 47.3927    ,  68.6686    ],
       [ 47.0735    ,  88.6581    ],
       [ 62.50385   , 111.508     ],
       [ 95.        ,  15.5543    ],
       [ 79.20635   ,  68.6759    ],
       [ 62.3636    ,  47.2189    ],
       [ 60.4845    ,  85.9622    ],
       [ 54.        ,  37.8863    ],
       [ 80.6349    ,  42.7237    ],
       [ 65.0033    ,  54.5338    ],
       [ 53.3079    , 155.174     ],
       [ 83.9346    ,  52.4893    ],
       [ 69.06925   ,  70.6457    ],
       [ 55.        ,  45.2189    ],
       [ 54.7907    , 162.204     ],
       [ 62.9338    , 153.805     ],
       [ 47.3565    ,  41.7627    ],
       [ 64.9257    , 129.385     ],
       [ 67.7202    , 117.259     ],
       [ 68.9859    ,  65.9749    ],
 

In [113]:
# ml_df

In [114]:
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# rf = RandomForestRegressor()
# # Random search of parameters, using 3 fold cross validation, 
# # search across 100 different combinations, and use all available cores
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# rf_random.fit(x, y)
# y_pred = rf_random.predict(x)
# print('Mean Absolute Error:', mean_absolute_error(y, y_pred))
# print('Mean Squared Error:', mean_squared_error(y, y_pred))
# print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, y_pred)))

In [115]:
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import GridSearchCV

# from sklearn.metrics import mean_squared_error, mean_absolute_error
# params = {
#     "svr__C": [1e0, 1e1, 1e2, 1e3,1e4,1e6],
#     'svr__kernel':['linear','poly', 'rbf', 'sigmoid']
# }
# pipeline_svr = make_pipeline(StandardScaler(),
#                          SVR())
# clf_svr= GridSearchCV(pipeline_svr, params, cv=3)

# clf_svr.fit(x, y)
# y_pred = clf_svr.predict(x)

# print('Mean Absolute Error:', mean_absolute_error(y, y_pred))
# print('Mean Squared Error:', mean_squared_error(y, y_pred))
# print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, y_pred)))